In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch
import numpy as np

# Load pre-trained DialoGPT model and tokenizer
model_name = "microsoft/DialoGPT-small"  # I am using smaller model to reduce memory usage
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Set model to evaluation mode and move to GPU if available
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model.to(device)
model.eval()

# Load a portion of the SQuAD dataset
print("Loading and examining the SQuAD dataset...")
squad_data = load_dataset("squad", split="train[:1000]")  # Increased data size to improve generalization  # Reduced data size to save memory

# Examine the dataset structure by printing a few examples
print("Dataset Example:")
print(squad_data[0])  # Print the first example to understand the structure

# Preprocess the SQuAD dataset to create prompt-response pairs in a more conversational format
def preprocess_data(examples):
    inputs = ["Can you tell me " + question.lower() + "?" for question in examples["question"]]  # Reformatted prompts to be more conversational
    targets = [answer["text"][0] if len(answer["text"]) > 0 else "no answer" for answer in examples["answers"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")  # Reduced max length to save memory
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids  # Match input max length
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]
    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing to the dataset
tokenized_squad = squad_data.map(preprocess_data, batched=True, remove_columns=squad_data.column_names)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./dialoGPT-finetuned-squad",
    per_device_train_batch_size=1,  # Reduced batch size to fit in memory
    gradient_accumulation_steps=4,  # Reduce gradient accumulation to lower memory usage
    num_train_epochs=1,  # Reduced number of epochs
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50,
    save_total_limit=1,
    fp16=False  # Disable mixed precision training to save memory
)

# Create data collator to handle padding dynamically
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Set model back to evaluation mode
model.eval()

# Start chatting with the model
print("DialoGPT: Hello! Let's chat. Type 'exit' to end the conversation.")
chat_history_ids = None
step = 0

while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    
    # Encode user input and add it to the chat history
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt').to(device)
    
    if chat_history_ids is not None:
        chat_history_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        chat_history_ids = new_input_ids
    
    # Limit the length of the chat history to prevent it from growing too large
    max_chat_history_length = 128  # Reduced max length to save memory
    if chat_history_ids.shape[-1] > max_chat_history_length:
        chat_history_ids = chat_history_ids[:, -max_chat_history_length:]
    
    # Generate a response using the model
    attention_mask = chat_history_ids.ne(tokenizer.pad_token_id).long().to(device)
    response_ids = model.generate(
        chat_history_ids,
        max_length=min(chat_history_ids.shape[-1] + 50, 128),  # Ensure the generated length does not exceed the model's max length
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=attention_mask,
        do_sample=True,
        top_k=30,  # Lower top_k to reduce randomness
        top_p=0.98,  # Increase top_p to include more probable tokens
        temperature=0.6,  # Further decrease temperature to improve coherence
        repetition_penalty=1.5,  # Increase repetition penalty to reduce repetitive responses
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(response_ids[:, chat_history_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Print the response
    print(f"DialoGPT: {response}")
    
    # Update the chat history
    chat_history_ids = torch.cat([chat_history_ids, response_ids[:, chat_history_ids.shape[-1]:]], dim=-1)

print("DialoGPT: Goodbye!")

Loading and examining the SQuAD dataset...
Dataset Example:
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [5

Step,Training Loss
10,11.999700
20,8.266100
30,8.538900
40,8.488600
50,9.431700
60,9.110400
70,8.232500
80,8.615500
90,9.059900
100,7.473700


DialoGPT: Hello! Let's chat. Type 'exit' to end the conversation.
